In [47]:
import pandas as pd
import fnmatch
import os
import glob
import csv
import codecs
import boto3
import io
from io import StringIO

csv_buffer = StringIO()

# Load Data from Local directory

In [51]:
# Outdated!!! Read from local files, however files are stored on S3 from March 2021

path = '/Users/matthiashugli/Virtualenvs/youth-base/youth-base/training-analysis/data/*.csv'

#files = [os.path.join(dirpath, f) for dirpath, dirnames, files in os.walk(path) for f in fnmatch.filter(files, '*.xlsx')]
#files = os.listdir(path)

df = pd.DataFrame()
for filename in glob.glob(path):
    print(filename)
    file = pd.read_csv(filename, header=1, delimiter=',', low_memory=False)
    file.insert(0, 'filename', filename)
    df = df.append(file)


/Users/matthiashugli/Virtualenvs/youth-base/youth-base/training-analysis/data/s1_core_trm_player_unit_skills_202104.csv
/Users/matthiashugli/Virtualenvs/youth-base/youth-base/training-analysis/data/s1_core_trm_player_unit_skills_202101.csv
/Users/matthiashugli/Virtualenvs/youth-base/youth-base/training-analysis/data/s1_core_trm_player_unit_skills_202102.csv
/Users/matthiashugli/Virtualenvs/youth-base/youth-base/training-analysis/data/s1_core_trm_player_unit_skills_202103.csv


## Cleanse Training Data

In [52]:
### Properly name columns and broad data table
df = df.drop(df.columns[[3]], axis=1)
df = df.rename(columns={'Unnamed: 0': 'team', 'Unnamed: 1': 'keeper'})
unpivot_df = pd.melt(df, id_vars=['filename', 'team', 'keeper'], var_name='skills', value_name='value')

In [53]:
### Set numeric values and rename value columns for pivot
trainings = unpivot_df

trainings['value'] = pd.to_numeric(trainings.value, errors='coerce')

trainings['skills'] = trainings.skills.apply(lambda x: x.replace('.1', '.units') if fnmatch.fnmatch(x, '*.1') else x + ".duration")

### Pivot DataFrame for all categories and create two columns for duration in minutes and number of trainings
trainings[['skills', 'entity']] = trainings.skills.apply(lambda x: pd.Series(str(x).split('.')))
trainings = trainings.pivot(index=['filename', 'team', 'keeper', 'skills'], columns='entity', values='value').reset_index()
trainings = trainings.dropna(subset=['duration', 'units'])

In [54]:
### Remove and rename club and skill details
trainings[['team', 'club']] = trainings.team.str.split(' BSC', 1, expand=True)
trainings[['team', 'club']] = trainings.team.str.split(' YB', 1, expand=True)
trainings[['category', 'topic']] = trainings.skills.apply(lambda x: pd.Series(str(x).split(':')) if fnmatch.fnmatch(x, ':') else pd.Series(str(x).split(' ', 1)))
trainings['category'] = trainings.category.str.replace(':', '')

In [55]:
# Date cleansing
trainings[['file', 'type']] = trainings.filename.apply(lambda x: pd.Series(str(x).split('.')))
trainings['yearmonth'] = trainings.file.str[-6:]
trainings['date'] = pd.to_datetime(trainings.yearmonth, format='%Y%m', errors='coerce').dropna()
trainings['year'] = pd.DatetimeIndex(trainings['date']).year
trainings['month'] = trainings.date.dt.month_name()

In [56]:
### Drop columns and reorder DataFrame
trainings['team'] = trainings.team.apply(lambda x: 'U21' if fnmatch.fnmatch(x, '1. Mannschaft') else x)
trainings = trainings.drop(columns=['filename', 'topic', 'club', 'file', 'type']).reset_index()
trainings = trainings.reindex(columns=['team', 'keeper', 'date', 'year', 'yearmonth', 'month', 'category', 'skills', 'units', 'duration'])

In [57]:
### Modify training unites as int
trainings['units'] = trainings.units.astype(int)

### Divide duration in mintues by 1440 to display hh:mm in report
# trainings['duration'] = trainings.duration / 1440

## Export Cleansed DataFrame to aws S3

In [58]:
trainings_export = trainings
trainings_export.to_csv('cleansing_trainings.csv', index=False)
#trainings_export.to_csv(csv_buffer, index=False)
#s3.Object(bucket_name.name, 'cleansing_trainings.csv').put(Body=csv_buffer.getvalue())
#print(s3.Object(bucket_name.name, 'cleansing_trainings.csv').put(Body=csv_buffer.getvalue()))

NameError: name 's3' is not defined

In [138]:
### Outdated AWS S3 not in use anymore
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import s3

### Monthly data is stored to subfolder in S3 Bucket, read and store filenames
bucket_name = s3.Bucket('training-minutes')

bucket_list = []
for file in bucket_name.objects.filter(Prefix = 'rawdata/'):
    file_key = file.key
    if file_key.find('.csv') != -1:
        bucket_list.append(file.key)
print(len(bucket_list))

3


In [46]:
### Read all files in Bucket and return as a DataFrame
df = pd.DataFrame()
for file_name in bucket_list:
    obj = s3.Object(bucket_name.name, file_name)
    data = obj.get()['Body'].read()
    file = pd.read_csv(io.BytesIO(data), header=1, delimiter=',', low_memory=False)
    file.insert(0, 'filename', file_name)

    df = df.append(file)

### Save DataFrame to S3 as CSV
#df.to_csv(csv_buffer)
#s3.Object(bucket_name.name, 'staging_trainings.csv').put(Body=csv_buffer.getvalue())
#s3.upload_file

NameError: name 'bucket_list' is not defined